In [1]:
import os, sys, json
sys.path.insert(1, './shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = "rg-realtime-mcp-demo-v5"
resource_group_location = "eastus2"

apim_sku = 'Basicv2'

# gpt-4o-realtime and gpt-4o-mini-realtime are only available for eastuse2 or swedencentral
openai_resources = [ {"name": "azure-openai-sc-priya", "location": "eastus2"}]
openai_deployment_name = "gpt-4o-realtime-preview"
openai_model_name = "gpt-4o-realtime-preview"
openai_model_version = "2024-12-17"
openai_model_sku = "GlobalStandard"
openai_model_capacity = 5
openai_api_version = "2024-10-01-preview"

build = 0
weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "src/weather/mcp-server"

servicenow_mcp_server_image = "servicenow-mcp-server"
servicenow_mcp_server_src = "src/servicenow/mcp-server"
servicenow_instance_name = "" # Add here the name of your ServiceNow instance, e.g. "businessname-dev". Leave empty if you don't want to use ServiceNow.

spotify_mcp_server_image = "spotify-mcp-server"
spotify_mcp_server_src = "src/spotify/mcp-server"

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 14:46:52.995972 


## Verify the Azure CLI and connected Azure subscription


In [2]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 14:46:57.265912 :2s]
👉🏽 Current user: priyakedia@microsoft.com
👉🏽 Tenant ID: 16b3c013-d300-468d-ac64-7eda0820b6d3
👉🏽 Subscription ID: 8cebb108-a4d5-402b-a0c4-f7556126277f
✅ Retrieved az account ⌚ 14:46:57.265912 :2s]
👉🏽 Current user: priyakedia@microsoft.com
👉🏽 Tenant ID: 16b3c013-d300-468d-ac64-7eda0820b6d3
👉🏽 Subscription ID: 8cebb108-a4d5-402b-a0c4-f7556126277f


## 2. Create Deployment using Bicep

This lab uses Bicep to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the main.bicep directly to try different configurations.

In [ ]:
! pip install azure-mgmt-apimanagement

In [ ]:
from azure.identity import DefaultAzureCredential

from azure.mgmt.apimanagement import ApiManagementClient

"""
# PREREQUISITES
    pip install azure-identity
    pip install azure-mgmt-apimanagement
# USAGE
    python api_management_deleted_services_purge.py

    Before run the sample, please set the values of the client ID, tenant ID and client secret
    of the AAD application as environment variables: AZURE_CLIENT_ID, AZURE_TENANT_ID,
    AZURE_CLIENT_SECRET. For more info about how to get the value, please see:
    https://docs.microsoft.com/azure/active-directory/develop/howto-create-service-principal-portal
"""


def main():
    client = ApiManagementClient(
        credential=DefaultAzureCredential(),
        subscription_id="8cebb108-a4d5-402b-a0c4-f7556126277f",
    )

    response = client.deleted_services.begin_purge(
        service_name="apim-dy7dbipcowwgk",
        location="eastus2",
    ).result()
    print(response)


# x-ms-original-file: specification/apimanagement/resource-manager/Microsoft.ApiManagement/stable/2024-05-01/examples/ApiManagementDeletedServicesPurge.json
if __name__ == "__main__":
    main()

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name},
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIModelSKU": { "value": openai_model_sku },
        "openAIModelCapacity": { "value": openai_model_capacity },
        "openAIAPIVersion": { "value": openai_api_version },
        "serviceNowInstanceName": { "value": servicenow_instance_name }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

## Get the deployment Outputs

Retrieve the required outputs from previous bicep deployment step


In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway URL')
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    if servicenow_instance_name:
        servicenow_containerapp_resource_name = utils.get_deployment_output(output, 'servicenowMCPServerContainerAppResourceName', 'servicenow Container App Resource Name')
    spotify_containerapp_resource_name = utils.get_deployment_output(output, 'spotifyMCPServerContainerAppResourceName', 'Spotify Container App Resource Name')

## Build and Deploy MCP Servers

In [ ]:
build = build + 1 # increment the build number

utils.run(f"az acr build --image {weather_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {weather_mcp_server_src}/Dockerfile {weather_mcp_server_src}/. --no-logs", 
         "Weather MCP Server image was successfully built", "Failed to build the Weather MCP Server image")
utils.run(f'az containerapp update -n {weather_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{weather_mcp_server_image}:v0.{build}"', 
         "Weather MCP Server deployment succeeded", "Weather MCP Server deployment failed")

if servicenow_instance_name:
    utils.run(f"az acr build --image {servicenow_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {servicenow_mcp_server_src}/Dockerfile {servicenow_mcp_server_src}/. --no-logs", 
            "ServiceNow MCP Server image was successfully built", "Failed to build the ServiceNow MCP Server image")
    utils.run(f'az containerapp update -n {servicenow_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{servicenow_mcp_server_image}:v0.{build}"', 
            "ServiceNow MCP Server deployment succeeded", "ServiceNow MCP Server deployment failed")

utils.run(f"az acr build --image {spotify_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {spotify_mcp_server_src}/Dockerfile {spotify_mcp_server_src}/. --no-logs", 
          "Spotify MCP Server image was successfully built", "Failed to build the Spotify MCP Server image")
utils.run(f'az containerapp update -n {spotify_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{spotify_mcp_server_image}:v0.{build}"', 
          "Spotify MCP Server deployment succeeded", "Spotify MCP Server deployment failed")

In [3]:
import os, json, asyncio, time, requests
from mcp import ClientSession
from mcp.client.sse import sse_client
import nest_asyncio
nest_asyncio.apply()

async def list_tools(server_url, authorization_header = None):
    headers = {"Authorization": authorization_header} if authorization_header else None
    async with sse_client(server_url, headers) as streams:
        async with ClientSession(streams[0], streams[1]) as session:
            await session.initialize()

            response = await session.list_tools()
            tools = response.tools
    print(f"✅ Connected to server {server_url}")
    print("⚙️ Tools:")
    for tool in tools:
        print(f"  - {tool.name}")
        print(f"     Input Schema: {tool.inputSchema}")

apim_resource_gateway_url = "https://apim-rydm3isbc5cmc.azure-api.net"   

asyncio.run(list_tools(f"{apim_resource_gateway_url}/spotify/mcp/sse"))
asyncio.run(list_tools(f"{apim_resource_gateway_url}/weather/sse"))

✅ Connected to server https://apim-rydm3isbc5cmc.azure-api.net/spotify/mcp/sse
⚙️ Tools:
  - authorize_spotify
     Input Schema: {'properties': {}, 'title': 'authorize_spotifyArguments', 'type': 'object'}
  - get_user_playlists
     Input Schema: {'properties': {}, 'title': 'get_user_playlistsArguments', 'type': 'object'}
  - get_player_queue
     Input Schema: {'properties': {}, 'title': 'get_player_queueArguments', 'type': 'object'}
  - get_playback_status
     Input Schema: {'properties': {}, 'title': 'get_playback_statusArguments', 'type': 'object'}
  - start_playback
     Input Schema: {'properties': {}, 'title': 'start_playbackArguments', 'type': 'object'}
  - pause_playback
     Input Schema: {'properties': {}, 'title': 'pause_playbackArguments', 'type': 'object'}
  - get_my_queue
     Input Schema: {'properties': {}, 'title': 'get_my_queueArguments', 'type': 'object'}
  - browse_new_releases
     Input Schema: {'properties': {}, 'title': 'browse_new_releasesArguments', 'type':

## Execute a Realtime Semantic Kernel Agent using MCP Tools via Azure API Management

In [7]:
from semantic_kernel.connectors.ai.open_ai import (
    AzureRealtimeWebsocket,
    AzureRealtimeExecutionSettings,
    ListenEvents,
    TurnDetection
)
from semantic_kernel.contents import RealtimeTextEvent
from semantic_kernel.contents.text_content import TextContent
from semantic_kernel.connectors.mcp import MCPSsePlugin

realtime_agent = AzureRealtimeWebsocket(
            endpoint=f"{apim_resource_gateway_url}/rt-audio",
            deployment_name=openai_deployment_name,
            azure_openai_realtime_deployment_name=openai_deployment_name,
            api_key=apim_subscription_key,
            api_version=openai_api_version)

weather_plugin = MCPSsePlugin(
    name="Weather",
    url=f"{apim_resource_gateway_url}/weather/sse",
    description="Weather Plugin",
)
    
settings = AzureRealtimeExecutionSettings(modalities=["text"], turn_detection=TurnDetection(type="server_vad", create_response=True, silence_duration_ms=800, threshold=0.8),

                                          instructions="You are a helpful assistant that provides information about the weather, using just the text modality",)
async with realtime_agent(settings=settings, create_response=True, plugins=[weather_plugin]) as connection:
    await connection.send(RealtimeTextEvent(text=TextContent(text="What's the weather like in Bangalore?")))

    async for event in connection.receive():
        if event.service_type == "response.text.done":
            print(event, flush=True, end="")
        elif event.service_type == "response.done":
            break

service_event=ResponseTextDoneEvent(content_index=0, event_id='event_BZwN3tWdKN2Ixrf9N29iX', item_id='item_BZwN3cxsuHsx4DEbDzr3Q', output_index=0, response_id='resp_BZwN3yfIVgimLyRMaElwL', text='Hello! How can I assist you today?', type='response.text.done') service_type='response.text.done'

## 5️⃣ Create a Spotify OAuth app and configure the credential provider
Step 1 - Create an account on Spotify Developer portal

Step 2 - Create an app with redirect URI from the next step
👉 Use the Authorization callback URL that is provided below
👉 Copy the Client ID and Client secret

Step 3 - Configure the credential provider in API Management
👉 You just need to update the Client ID and Client secret on the existing spotify credential manager provider

In [ ]:
print(f"Authorization callback URL: https://authorization-manager.consent.azure-apim.net/redirect/apim/{apim_resource_name}")

Authorization callback URL: https://authorization-manager.consent.azure-apim.net/redirect/apim/apim-rydm3isbc5cmc


Test the Realtime API using FastRTC + Gradio

FastRTC is an elegant realtime library communication library to enable you to easily and quickly build RTC application both using websockets and WebRTC.

Please ensure you have run the pip command succefully to install all required packages

👉 Tip: Restart the Python Kernel to stop the FastRTC server

In [ ]:
import asyncio, base64, json, random, openai
import gradio as gr
import numpy as np
from openai.types.beta.realtime import ResponseAudioTranscriptDoneEvent, ConversationItemCreateEvent
from fastrtc import (
    AdditionalOutputs,
    AsyncStreamHandler,
    Stream,
    wait_for_item,
    UIArgs
)
from gradio.utils import get_space
from mcp_client import OAI_RT_SSEMCPClient

# Define the sample rate (Hz)
SAMPLE_RATE = 24000

AZURE_OPENAI_API_ENDPOINT = apim_resource_gateway_url + "/rt-audio"
AZURE_OPENAI_API_KEY = apim_subscription_key
AZURE_OPENAI_API_VERSION = openai_api_version
AZURE_OPENAI_DEPLOYMENT_NAME = openai_deployment_name

class OpenAIHandler(AsyncStreamHandler):
    mcp_config = {
        'spotify': f"{apim_resource_gateway_url}/spotify/mcp/sse",
        'weather': f"{apim_resource_gateway_url}/weather/sse"
        }
    mcp_servers = {}
    tool_server_map = {}

    def __init__(self) -> None:
        super().__init__(
            expected_layout="mono",
            output_sample_rate=SAMPLE_RATE,
            output_frame_size=480,  # In this example we choose 480 samples per frame.
            input_sample_rate=SAMPLE_RATE,
        )
        self.connection = None
        if len(self.mcp_config) > 0:
            print ("⚙️ Initializing all MCPs")
            for name, url in self.mcp_config.items():
                self.mcp_servers[name] = OAI_RT_SSEMCPClient(server_name=name, url=url)
        else:
            print ("no MCPs")
        self.output_queue = asyncio.Queue()

    async def start_up(self):
        """
        Establish a persistent realtime connection to the Azure OpenAI backend.
        The connection is configured for server‐side Voice Activity Detection.
        """
        self.client = openai.AsyncAzureOpenAI(
            azure_endpoint=AZURE_OPENAI_API_ENDPOINT,
            azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION,
        )
        for name, server in self.mcp_servers.items():
                await server.start()
        # When using Azure OpenAI realtime (beta), set the model/deployment identifier
        async with self.client.beta.realtime.connect(
            model=AZURE_OPENAI_DEPLOYMENT_NAME  # Replace with your deployed realtime model id on Azure OpenAI.
        ) as conn:
            # Configure the session to use server-based voice activity detection (VAD)
            session_config = await self.session_config()
            await conn.session.update(session=session_config) # type: ignore
            self.connection = conn

            # Uncomment the following line to send a welcome message to the assistant.
            # await self.welcome()

            async for event in self.connection:
                # Handle interruptions
                if event.type == "input_audio_buffer.speech_started":
                    self.clear_queue()
                if event.type == "conversation.item.input_audio_transcription.completed":
                    # This event signals that an input audio transcription is completed.
                    await self.output_queue.put(AdditionalOutputs(event))
                if event.type == "response.audio_transcript.done":
                    # This event signals that a response audio transcription is completed.
                    await self.output_queue.put(AdditionalOutputs(event))
                if event.type == "response.function_call_arguments.done":
                    await self.get_tool_response(event)
                if event.type == "conversation.item.created":
                    await self.output_queue.put(AdditionalOutputs(event))
                if event.type == "response.audio.delta":
                    # For incremental audio output events, decode the delta.
                    await self.output_queue.put(
                        (
                            self.output_sample_rate,
                            np.frombuffer(base64.b64decode(event.delta), dtype=np.int16).reshape(1, -1),
                        ),
                    )

    def copy(self):
        return OpenAIHandler()

    async def welcome(self):
        await self.connection.conversation.item.create( # type: ignore
            item={
                "type": "message",
                "role": "user",
                "content": [{"type": "input_text", "text": "what's your name?"}],
            }
        )
        await self.connection.response.create() # type: ignore

    async def receive(self, frame: tuple[int, np.ndarray]) -> None:
        """
        Receives an audio frame from the stream and sends it into the realtime API.
        The audio data is encoded as Base64 before appending to the connection's input.
        """
        if not self.connection:
            return
        _, array = frame
        array = array.squeeze()
        # Encode audio as Base64 string
        audio_message = base64.b64encode(array.tobytes()).decode("utf-8")
        await self.connection.input_audio_buffer.append(audio=audio_message)  # type: ignore

    async def emit(self) -> tuple[int, np.ndarray] | AdditionalOutputs | None:
        """
        Waits for and returns the next output from the output queue.
        The output may be an audio chunk or an additional output such as transcription.
        """
        return await wait_for_item(self.output_queue)

    async def shutdown(self) -> None: # type: ignore
        # await self.weather_mcp.stop()
        if self.connection:
            await self.connection.close()
            self.connection = None

    async def session_config(self):
        """Returns a random value from the predefined list."""
        values = ['alloy', 'ash', 'ballad', 'coral', 'echo', 'sage', 'shimmer', 'verse']
        tools = []
        ### Get all tools from all active servers
        for name, server in self.mcp_servers.items():
                current_server_tools = await server.list_tools()
                for tool in current_server_tools:
                    self.tool_server_map[tool['name']]= name
                tools = tools + current_server_tools
        print(self.tool_server_map)
        voice = random.choice(values)
        print(voice)
        ### for details on available param: https://platform.openai.com/docs/api-reference/realtime-sessions/create
        SESSION_CONFIG={
            "input_audio_transcription": {
                "model": "whisper-1",
            },
            "turn_detection": {
                "threshold": 0.4,
                "silence_duration_ms": 600,
                "type": "server_vad"
            },
            "instructions": f"""You are a DJ named {voice}! 
                                You are a helpful, calm and cheerful agent who responds with a clam accent, but also can speak in any language or accent. 
                                If you need to call a spotify tool, inform the user that you need first to call the authorization tool.
                                After calling the authorization tool, send a text event with the link URL and don't read the link.
                                After sending the event with the link, inform the user that he just needs to click on the provide link to get authorized on Spotify.""",
            "voice": voice,
            "modalities": ["text", "audio"], ## required to solicit the initial welcome message
            "tools": tools
        }
        return SESSION_CONFIG

    async def get_tool_response(self, event):
        ### findout which MCP server to call
        server_name = self.tool_server_map[event.name]
        ### call the target
        response = await self.mcp_servers[server_name].call_tool(tool_call=event.model_dump())
        await self.connection.conversation.item.create(item=response) # type: ignore
        await self.connection.response.create() # type: ignore

def on_open(ws):
    print("Connected to server.")

def on_message(ws, message):
    data = json.loads(message)
    print("Received event:", json.dumps(data, indent=2))

def update_chatbot(chatbot: list[dict], event):
    """
    Append the completed transcription to the chatbot messages.
    """
    if event.type == "conversation.item.input_audio_transcription.completed":
        chatbot.append({"role": "user", "content": event.transcript})
    elif event.type == "response.audio_transcript.done":
        chatbot.append({"role": "assistant", "content": event.transcript})
    elif event.type == "conversation.item.created":
        if event.item and event.item.output:
            output = str(event.item.output)
            expected_output = "Please authorize by opening this link:" # this string must match what was coded in the mcp server
            if expected_output in output:
                link = f'<a href="{output.replace(expected_output, "").strip()}">👉 Click here to authorize</a>'
                chatbot.append({"role": "assistant", "content": link})
    return chatbot

ui_args: UIArgs = UIArgs(
    title="APIM ❤️ MCP - DJ Contoso Assistant 🎧",
)
chatbot = gr.Chatbot(type="messages")
latest_message = gr.Textbox(type="text", visible=True)

# Instantiate the Stream object that uses the OpenAIHandler.
stream = Stream(
    OpenAIHandler(),
    mode="send-receive",
    modality="audio",
    additional_inputs=[chatbot],
    additional_outputs=[chatbot],
    additional_outputs_handler=update_chatbot,
    ui_args=ui_args,
)

if __name__ == "__main__":
    stream.ui.launch(server_port=7860)

c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⚙️ Initializing all MCPs
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


⚙️ Initializing all MCPs
{'authorize_spotify': 'spotify', 'get_user_playlists': 'spotify', 'get_player_queue': 'spotify', 'get_playback_status': 'spotify', 'start_playback': 'spotify', 'pause_playback': 'spotify', 'get_my_queue': 'spotify', 'browse_new_releases': 'spotify', 'search': 'spotify', 'check_authorization_status': 'spotify', 'check_and_repair_authorization': 'spotify', 'verify_authorization_with_test_call': 'spotify', 'test_connection': 'spotify', 'check_environment_variables': 'spotify', 'get_cities': 'weather', 'get_weather': 'weather'}
echo


Traceback (most recent call last):
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\fastrtc\utils.py", line 474, in async_wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\priyakedia\AppData\Local\Temp\ipykernel_32196\1921268966.py", line 72, in start_up
    async for event in self.connection:
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\openai\resources\beta\realtime\realtime.py", line 258, in __aiter__
    yield await self.recv()
          ^^^^^^^^^^^^^^^^^
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\openai\resources\beta\realtime\realtime.py", line 268, in recv
    return self.parse_event(await self.recv_bytes())
                            ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\openai\resources\beta

⚙️ Initializing all MCPs
{'authorize_spotify': 'spotify', 'get_user_playlists': 'spotify', 'get_player_queue': 'spotify', 'get_playback_status': 'spotify', 'start_playback': 'spotify', 'pause_playback': 'spotify', 'get_my_queue': 'spotify', 'browse_new_releases': 'spotify', 'search': 'spotify', 'check_authorization_status': 'spotify', 'check_and_repair_authorization': 'spotify', 'verify_authorization_with_test_call': 'spotify', 'test_connection': 'spotify', 'check_environment_variables': 'spotify', 'get_cities': 'weather', 'get_weather': 'weather'}
verse


Exception ignored in: <async_generator object AsyncClient.stream at 0x000001FCA9154CC0>
Traceback (most recent call last):
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\httpx\_client.py", line 1592, in stream
    await response.aclose()
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\httpx\_models.py", line 1076, in aclose
    await self.stream.aclose()
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\httpx\_client.py", line 182, in aclose
    await self._stream.aclose()
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\httpx\_transports\default.py", line 276, in aclose
    await self._httpcore_stream.aclose()
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\httpcore\_async\connection_pool.py", line 412, in aclose
    with AsyncShieldCancellation(

⚙️ Initializing all MCPs
{'authorize_spotify': 'spotify', 'get_user_playlists': 'spotify', 'get_player_queue': 'spotify', 'get_playback_status': 'spotify', 'start_playback': 'spotify', 'pause_playback': 'spotify', 'get_my_queue': 'spotify', 'browse_new_releases': 'spotify', 'search': 'spotify', 'check_authorization_status': 'spotify', 'check_and_repair_authorization': 'spotify', 'verify_authorization_with_test_call': 'spotify', 'test_connection': 'spotify', 'check_environment_variables': 'spotify', 'get_cities': 'weather', 'get_weather': 'weather'}
echo


Exception ignored in: <async_generator object sse_client at 0x000001FCA9136880>
Traceback (most recent call last):
  File "C:\Users\priyakedia\AppData\Local\Temp\ipykernel_32196\1921268966.py", line 42, in __init__
RuntimeError: async generator ignored GeneratorExit
Exception ignored in: <async_generator object sse_client at 0x000001FCA9135D00>
Traceback (most recent call last):
  File "C:\Users\priyakedia\AppData\Local\Temp\ipykernel_32196\1921268966.py", line 42, in __init__
RuntimeError: async generator ignored GeneratorExit


⚙️ Initializing all MCPs
{'authorize_spotify': 'spotify', 'get_user_playlists': 'spotify', 'get_player_queue': 'spotify', 'get_playback_status': 'spotify', 'start_playback': 'spotify', 'pause_playback': 'spotify', 'get_my_queue': 'spotify', 'browse_new_releases': 'spotify', 'search': 'spotify', 'check_authorization_status': 'spotify', 'check_and_repair_authorization': 'spotify', 'verify_authorization_with_test_call': 'spotify', 'test_connection': 'spotify', 'check_environment_variables': 'spotify', 'get_cities': 'weather', 'get_weather': 'weather'}
coral


Traceback (most recent call last):
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\fastrtc\utils.py", line 474, in async_wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\priyakedia\AppData\Local\Temp\ipykernel_32196\1921268966.py", line 83, in start_up
    await self.get_tool_response(event)
  File "C:\Users\priyakedia\AppData\Local\Temp\ipykernel_32196\1921268966.py", line 170, in get_tool_response
    server_name = self.tool_server_map[event.name]
                  ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'add_to_playlist'
Traceback (most recent call last):
  File "c:\Users\priyakedia\projects\realtime-audio-with-mcp-integration\.venv\Lib\site-packages\fastrtc\utils.py", line 474, in async_wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\priyakedia\AppData\Local\Temp\ipykernel_32196\1921268966.py", line 119, in receive
    awai